# Model Deployment
## Azure ML and MLFlow tracking

In [ ]:
# Deploy model
# Connect to workspace
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()
ml_client = MLClient(
    credential=credential,
    subscription_id="...",
    resource_group_name="...",
    workspace_name="...",
)

In [ ]:
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
import os

score_dir = './score'
os.makedirs(score_dir, exist_ok=True)

In [ ]:
%%writefile {score_dir}/score.py
import os
import logging
import json
import numpy
import joblib
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import requests
from datetime import datetime
FN_URL = 'https://<your fn app name>.azurewebsites.net/api'

def init():
    """
    This function is called when the container is initialized/started, typically after create/update of the deployment.
    You can write the logic here to perform init operations like caching the model in memory
    """
    global model
    # AZUREML_MODEL_DIR is an environment variable created during deployment.
    # It is the path to the model folder (./azureml-models/$MODEL_NAME/$VERSION)
    model_path = os.path.join(
        os.getenv("AZUREML_MODEL_DIR"), "model/model.pkl"
    )
    print(model_path)
    logging.info("Loading model from "+model_path)
    # deserialize the model file back into a sklearn model
    model = joblib.load(model_path)
    logging.info("Init complete")


def run(raw_data):
    """
    This function is called for every invocation of the endpoint to perform the actual scoring/prediction.
    In the example we extract the data from the json input and call the scikit-learn model's predict()
    method and return the result back
    """
    logging.info("Request received")
    arr = json.loads(raw_data)
    payload = []
    results = []
    for data in arr:
        id = data['id']
        ts = data['ts']
        inputs = data['inputs']
        npdata = numpy.array(inputs)
        value = model.predict(npdata)[0]
        results.append(value)
        payload.append({"id":id,"ts":ts, "inputs":json.dumps(inputs), "value":value, "type":"prediction"})
    logging.info("Request processed")
    requests.post(FN_URL+'/DataLogger',json=payload)
    return results


In [ ]:
%%writefile {score_dir}/conda.yml
name: model-env
channels:
  - conda-forge
dependencies:
  - python=3.8
  - numpy=1.21.2
  - pip=21.2.4
  - scikit-learn=0.24.2
  - scipy=1.7.1
  - pandas>=1.1,<1.2
  - pip:
    - azureml-defaults>=1.42.0
    - azureml-inference-server-http
    - inference-schema[numpy-support]==1.3.0
    - xlrd==2.0.1
    - mlflow== 1.26.1
    - azureml-mlflow==1.42.0
    - psutil>=5.8,<5.9
    - tqdm>=4.59,<4.60
    - ipykernel~=6.0
    - matplotlib

In [ ]:
from azure.ai.ml.entities import Environment

custom_env_name = "bike-share-env"

env = Environment(
    name=custom_env_name,
    description="Custom environment for Bike Share pipeline",
    tags={"scikit-learn": "0.24.2"},
    conda_file= "score/conda.yml",
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:latest",
)
env = ml_client.environments.create_or_update(env)

print(
    f"Environment {env.name} registered to workspace. "
    f"Environment version is {env.version}."
)

## Endpoints and Deployments

- An Endpoint is an HTTP server that allows interaction with the model(s).
- An endpoint may serve multiple models.
- Each model must be deployed in order for the endpoint to serve it.
- Hence, one Endpoint can have many Deployments.
- Blue-Green deployment and traffic management can be handled here or using az CLI.

## Creating Endpoint
### Python SDK

In [ ]:
# Create an endpoint name
endpoint_name = "my-endpoint"

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name = endpoint_name, 
    description="Bike share endpoint",
    auth_mode="key",
)

ml_client.online_endpoints.begin_create_or_update(endpoint, ).result()

### Azure CLI 
You can achieve the same by running 
```
az ml online-endpoint create --file path/to/my-endpoint.yml --resource-group ... --workspace-name ...
```
where `my-endpoint.yml` is as follows:
```yml
$schema: https://azuremlschemas.azureedge.net/latest/managedOnlineEndpoint.schema.json
name: my-endpoint
description: Bike share endpoint
auth_mode: key
```

More info: [here](https://learn.microsoft.com/en-us/cli/azure/ml/online-deployment?view=azure-cli-latest)

## Creating a Deployment
### Python SDK

In [ ]:
# grab the model to be deployed and the serving environment
model = ml_client.models.get('sklearn.ensemble._forest.RandomForestRegressor',1)
env = ml_client.environments.get('bike-share-env', 1)

deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(
        code="./score", scoring_script="score.py"
    ),
    instance_type="Standard_DS3_v2",
    instance_count=1,
    app_insights_enabled=True,
)
ml_client.online_deployments.begin_create_or_update(deployment=deployment, ).result()

### Azure CLI

You can use
```sh
az ml online-deployment create --file path/to/deployment.yaml --resource-group ... --workspace-name ...
```
where `deployment.yml` is as follows:
```yml
$schema: https://azuremlschemas.azureedge.net/latest/managedOnlineDeployment.schema.json
name: blue
endpoint_name: my-endpoint
model: azureml:sklearn.ensemble._forest.RandomForestRegressor:1
environment: azureml:bike-share-env:1
code_configuration:
  code: score/
  scoring_script: score.py
instance_type: Standard_DS3_v2
instance_count: 1
app_insights_enabled: true
```

More info: [here](https://learn.microsoft.com/en-us/cli/azure/ml/online-endpoint?view=azure-cli-latest)